<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 02 - Data Virtualization

In this tutorial you will learn how to create and query Virtualized Data in a SQL Server big data cluster. 

**TODO:** Complete Tutorial 


In [1]:
/* Create External File Format */

USE WideWorldImporters;
GO

IF NOT EXISTS(SELECT * FROM sys.external_file_formats WHERE name = 'csv_file')
BEGIN
    CREATE EXTERNAL FILE FORMAT csv_file
    WITH (
        FORMAT_TYPE = DELIMITEDTEXT,
        FORMAT_OPTIONS(
            FIELD_TERMINATOR = ',',
            STRING_DELIMITER = '0x22',
            FIRST_ROW = 2,
            USE_TYPE_DEFAULT = TRUE)
    );
END

Commands completed successfully.

Total execution time: 00:00:00.0788445

Commands completed successfully.

Total execution time: 00:00:00.8864665

In [2]:
/* Create External Data Source to the Storage Pool */
IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlStoragePool')
BEGIN
    CREATE EXTERNAL DATA SOURCE SqlStoragePool
    WITH (LOCATION = 'sqlhdfs://service-master-pool:50070');
END

Commands completed successfully.

Total execution time: 00:00:00.3917940

In [3]:
/* Create an External Table that can read from the Storage Pool File Location */
IF NOT EXISTS(SELECT * FROM sys.external_tables WHERE name = 'partner_customers_hdfs')
BEGIN
    CREATE EXTERNAL TABLE [partner_customers_hdfs]
    ("CustomerSource" VARCHAR(250) 
    , "CustomerName" VARCHAR(250) 
    , "EmailAddress" VARCHAR(250))
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/partner_customers',
        FILE_FORMAT = csv_file
    );
END

Commands completed successfully.

Total execution time: 00:00:00.7815709

In [1]:
/* Read Data from HDFS using only T-SQL */

SELECT TOP 10 CustomerSource
, CustomerName
, EMailAddress
    FROM [partner_customers_hdfs] hdfs
WHERE EmailAddress LIKE '%wingtip%'
ORDER BY CustomerSource, CustomerName;
GO


(10 rows affected)

Total execution time: 00:00:02.5322630

CustomerSource,CustomerName,EMailAddress
AdventureWorks,Ã…Å¡ani Nair,Ã¥Å¡ani@wingtiptoys.com
AdventureWorks,Ã…Å¡ani Sen,Ã¥Å¡ani@wingtiptoys.com
AdventureWorks,Aakriti Bhamidipati,aakriti@wingtiptoys.com
AdventureWorks,Aamdaal Kamasamudram,aamdaal@wingtiptoys.com
AdventureWorks,Abel Pirvu,abel@wingtiptoys.com
AdventureWorks,Abhaya Rambhatla,abhaya@wingtiptoys.com
AdventureWorks,Abhra Thakur,abhra@wingtiptoys.com
AdventureWorks,Adam Balaz,adam@wingtiptoys.com
AdventureWorks,Adirake Narkbunnum,adirake@wingtiptoys.com
AdventureWorks,Adirake Saenamuang,adirake@wingtiptoys.com


In [2]:
/* Now Join Those to show customers we currently have in a SQL Server Database 
and the Category they qre in the External Table */
USE WideWorldImporters;
GO

SELECT TOP 10 a.FullName
  , b.CustomerSource
  FROM Application.People a
  INNER JOIN partner_customers_hdfs b  ON a.FullName = b.CustomerName
  ORDER BY FullName ASC;
  GO

Commands completed successfully.

Total execution time: 00:00:00.0762610

(10 rows affected)

Total execution time: 00:00:02.0456260

FullName,CustomerSource
Aahlada Thota,AdventureWorks
Aakarsha Nookala,AdventureWorks
Aakriti Bhamidipati,AdventureWorks
Aamdaal Kamasamudram,AdventureWorks
Abel Pirvu,AdventureWorks
Abhaya Rambhatla,AdventureWorks
Abhra Thakur,AdventureWorks
Adam Balaz,AdventureWorks
Adam Dvorak,AdventureWorks
Adam Kubat,AdventureWorks


## Next Steps: Continue on to Working with the SQL Server Data Pool